In [ ]:
import pandas as pd

In [ ]:
!pip install autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=824a3e15036280b53cfc0ad619de990b300303f64f8fab15ac7cc7cbcfba4d3a
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


In [ ]:
from autocorrect import Speller
spell = Speller(lang='en')

In [ ]:
df = pd.read_csv('jamcrime.csv')

In [ ]:
df.head()

,CRIMEID,DATE,CRIME_TYPE,LOCATION,NUMBER_OF_VICTIMS
0,CID0035,2023-06-19,Robbery,Montego Bay,3.0
1,CID0329,2023-05-17,Assault,Negril,4.0
2,CID0857,2023-10-05,Burglary,Ocho Rios,3.0
3,CID0434,2023-03-10,Homicide,Ocho Rios,3.0
4,CID0323,2023-08-15,Burglary,Port Antonio,3.0


In [ ]:
len(df)

1050

A good habit is to remove trailing spaces

In [ ]:
df.columns = [col.strip() for col in df.columns]

In [ ]:
df.head()

,CRIMEID,DATE,CRIME_TYPE,LOCATION,NUMBER_OF_VICTIMS
0,CID0035,2023-06-19,Robbery,Montego Bay,3.0
1,CID0329,2023-05-17,Assault,Negril,4.0
2,CID0857,2023-10-05,Burglary,Ocho Rios,3.0
3,CID0434,2023-03-10,Homicide,Ocho Rios,3.0
4,CID0323,2023-08-15,Burglary,Port Antonio,3.0


In [ ]:
len(df)

1050

In [ ]:
df.isnull().values.any()

True

In [ ]:
df['CRIMEID'].isnull().values.any()

False

In [ ]:
df['DATE'].isnull().values.any()

False

In [ ]:
df['DATE'] = pd.to_datetime(df['DATE']).dt.date

<ipython-input-13-cbc38ff600bd>:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['DATE'] = pd.to_datetime(df['DATE']).dt.date


In [ ]:
df.dropna(subset=['DATE'], inplace = True)

In [ ]:
len(df)

1050

In [ ]:
df['CRIME_TYPE'].isnull().values.any()

False

In [ ]:
def correct_spelling(text):
    text = text.replace('Frad', 'Fraud')
    text = text.replace('Burgulary', 'Burglary')
    return spell(text)

In [ ]:
df['CRIME_TYPE'] = df['CRIME_TYPE'].apply(correct_spelling)

In [ ]:
df['LOCATION'].isnull().values.any()

True

In [ ]:
null_location_count = df['LOCATION'].isnull().sum()
print('The number of null values in the LOCATION column is',null_location_count)

The number of null values in the LOCATION column is 50


In [ ]:
df.dropna(subset=['LOCATION'], inplace=True)

In [ ]:
len(df)

1000

In [ ]:
df['NUMBER_OF_VICTIMS'].isnull().values.any()

True

In [ ]:
null_victims_count = df['NUMBER_OF_VICTIMS'].isnull().sum()
print('The number of null values in the NUMBER_OF_VICTIMS column is',null_victims_count)

The number of null values in the NUMBER_OF_VICTIMS column is 9


In [ ]:
df.dropna(subset=['NUMBER_OF_VICTIMS'], inplace=True)

In [ ]:
len(df)

991

In [ ]:
duplicate_rows=df.duplicated()

In [ ]:
num_duplicates = duplicate_rows.sum()
print(f'Number of duplicated rows: {num_duplicates}')

Number of duplicated rows: 47


In [ ]:
df = df.drop_duplicates()

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
len(df)

944

In [ ]:
num_negative_rows = (df['NUMBER_OF_VICTIMS'] < 0).sum()
print(f'Number of rows with negative values in the "NUMBER_OF_VICTIMS" column before filtering: {num_negative_rows}')

Number of rows with negative values in the "NUMBER_OF_VICTIMS" column before filtering: 4


In [ ]:
df = df[df['NUMBER_OF_VICTIMS'] >= 0]

In [ ]:
len(df)

940

In [ ]:
Q1 = df['NUMBER_OF_VICTIMS'].quantile(0.25)
Q3 = df['NUMBER_OF_VICTIMS'].quantile(0.75)
IQR = Q3 - Q1

In [ ]:
threshold = 1.5
outliers = df[(df['NUMBER_OF_VICTIMS'] < Q1 - threshold * IQR) | (df['NUMBER_OF_VICTIMS'] > Q3 + threshold * IQR)]

In [ ]:
num_outliers = len(outliers)
print(f'The number of outliers is {num_outliers}')

The number of outliers is 51


In [ ]:
print(outliers)

     CRIMEID        DATE CRIME_TYPE      LOCATION  NUMBER_OF_VICTIMS
29   CID0189  2023-03-03   Homicide  Port Antonio               48.0
58   CID0405  2023-11-01      Fraud        Negril               20.0
95   CID0830  2023-09-09   Homicide  Port Antonio               38.0
107  CID0077  2023-01-28   Burglary     Ocho Rios               24.0
176  CID0119  2023-04-18    Assault        Negril               26.0
180  CID0311  2023-03-07    Assault        Negril               23.0
186  CID0719  2023-10-10      Fraud  Port Antonio               24.0
200  CID0714  2023-01-26   Homicide  Port Antonio               29.0
225  CID0979  2023-06-08    Robbery  Port Antonio               25.0
235  CID0083  2023-10-15   Burglary   Montego Bay               33.0
238  CID0628  2023-03-10   Homicide      Kingston               31.0
253  CID0276  2023-01-22    Assault     Ocho Rios               35.0
263  CID0352  2023-09-08      Fraud  Port Antonio               22.0
274  CID0523  2023-08-04    Assaul

In [ ]:
df = df.drop(outliers.index)

In [ ]:
len(df)

889

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.to_excel('PJ_Thompson_cleaned_crime_data.xlsx', index=False)

In [ ]:
df.to_csv('PJ_Thompson_cleaned_crime_data.csv', index=False)